In [1]:
import pandas as pd

df = pd.read_csv('./merged.csv')

In [2]:
print(df.head())

  artist_names  track_name                                             lyrics  \
0          IVE        I AM  다른 문을 열어, 따라갈 필요는 없어 넌 너의 길로, 난 나의 길로, mm 하루하루...   
1          IVE      Kitsch  It's our time 우린 달라, 특별한 게 좋아 Oh, what a good ...   
2        Jimin  Like Crazy  She's saying, "Baby, 생각하지 마" There's not a bad...   
3        JISOO      FLOWER  Eh-eh-eh-eh Eh-eh-eh-eh A-B-C, Do-Re-Mi 만큼 착했던...   
4     NewJeans         OMG  이 노래는 it's about you, baby Only you You, you, ...   

       r      g      b  
0   41.0   41.0   41.0  
1   41.0   41.0   41.0  
2  156.0  152.0  151.0  
3   39.0   33.0    8.0  
4  195.0  195.0  195.0  


## SentenceTransformer 이용

In [2]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

embedder = SentenceTransformer('jhgan/ko-sroberta-multitask')

In [98]:
corpus = df['lyrics'].tolist()

In [5]:
# SBERT를 이용해 문장 임베딩


corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

In [4]:
# 임베딩에 시간이 꽤 많이 소요되므로 결과 텐서를 파일로 저장해둠.
import torch
torch.save(corpus_embeddings, "sbert_embeddings.pt")

NameError: name 'corpus_embeddings' is not defined

In [28]:
import torch
corpus_embeddings = torch.load("sbert_embeddings.pt")

In [113]:
query = '여름이라 너무 덥다'
query_embeddings = embedder.encode(query, convert_to_tensor=True)

cos_scores = util.pytorch_cos_sim(query_embeddings, corpus_embeddings)[0]
cos_scores = cos_scores.cpu()

In [114]:
top_results = np.argpartition(-cos_scores, 1)[:1]
print(df.iloc[top_results])

     artist_names  track_name  \
1381        SUNMI  Heart Burn   

                                                 lyrics      r      g      b  
1381  그대 아무런 말도 하지 마요 이 맘은 여전히 그대로예요 따가운 햇살 그 아래 우리 ...  223.0  162.0  118.0  


## Word2Vec 이용

In [4]:
from nltk.tokenize import word_tokenize
import nltk

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/cakenpeace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [48]:
pip install konlpy

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 32.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 583.3/583.3 kB 24.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
from konlpy.tag import Okt

In [6]:
from tqdm import tqdm

In [51]:
import re
for i in range (len(corpus)):
    corpus[i] = corpus[i].replace('(', '').replace(')', '').replace(',', '').replace('-','')

In [52]:
tokenized_data = []
for sent in tqdm(corpus):
    tokenised_sent = Okt().morphs(sent, stem=True)
    tokenized_data.append(tokenised_sent)  

100%|██████████| 1674/1674 [00:16<00:00, 99.03it/s] 


In [10]:
print(tokenized_data[0])

['다른', '문', '을', '열다', '따라가다', '필요', '는', '없다', '넌', '너', '의', '길', '로', '난', '나', '의', '길', '로', '하루하루', '마다', '색', '이', '달라지다', '느낌', '밝다', '빛', '이', '나', '는', '길', '을', '찾다', '넌', '그냥', '믿다', '돼다', '보이다', '그대로', '야', '너', '는', '누군가', '의', '제일', '좋다', '어느', '날', '의', '머물다', '고프다', '어딘가', '의', '낯선', '아름답다', '장르', '로는', '내일', '내다', '열리다', '건', '어제', '랑', '또', '다른', '짜릿하다', '나', '두렵다', '모든', '게', '설레다', '사소하다', '건', '다', '아득해지다', '오다', '넌', '그냥', '믿다', '돼다', '보이다', '그대로', '야', '너', '는', '누군가', '의', '제일', '좋다', '어느', '날', '의', '머물다', '고프다', '어딘가', '의', '낯선', '아름답다', '장르', '로는', '내일', '내다', '열리다', '건', '어느', '깊다', '밤', '길', '을', '잃다', '차라리', '날아오르다', '그렇다', '네', '가', '지나가다', '대로', '길이', '거들다', '제일', '좋다', '어느', '날', '의', '머물다', '고프다', '어딘가', '의', '낯선', '아름답다', '아름답다', '장르', '로는', '내일', '내다', '열리다', '건']


In [53]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

model = Word2Vec(sentences=tokenized_data, vector_size=100, window=5, min_count=1, workers=4)

In [56]:
model_result = model.wv.most_similar("눈사람")
print(model_result)

[('재난', 0.5594123005867004), ('Searching', 0.5531819462776184), ('힘없다', 0.5318965911865234), ('어나', 0.5281102061271667), ('stilettos', 0.5194351673126221), ('자락', 0.5164238214492798), ('푸르다', 0.5160735249519348), ('열렬하다', 0.5137729048728943), ('chitty', 0.5132858753204346), ('모래성', 0.513283908367157)]


In [84]:
model.wv.save_word2vec_format('w2v_nopre')

In [57]:
w2v_nopre_vec = []

In [59]:
from tqdm import tqdm, trange
for i in trange(len(tokenized_data)):
    sum = 0
    for word in tokenized_data[i]:
        sum += model.wv[word]
    sum /= len(tokenized_data[i])
    w2v_nopre_vec.append(sum)

100%|██████████| 1674/1674 [00:00<00:00, 3021.97it/s]


In [60]:
print(model.wv.index_to_key)

["'", 'I', 'you', 'the', 'me', 'it', 't', 'my', 'a', 's', 'to', 'm', '이', 'yeah', '?', '하다', 'that', 'in', '을', 'like', 'and', '에', 'on', '너', '내', 'your', 'You', 'love', 'know', '의', 'up', 'we', 'can', 'oh', 'don', '가', 'be', 'for', '나', 'all', 'of', 'go', 're', '난', '보다', 'just', 'And', 'with', 'so', 'no', '없다', 'll', '은', 'is', 'got', '있다', '를', 'get', 'what', 'baby', 'wanna', 'now', '로', '는', 'do', '날', 'this', 'out', '더', 'want', 'Oh', 'ooh', 'let', '그', '도', '네', '"', 'Yeah', 'But', '말', '다', 'time', 'was', 'one', 'not', 'when', 'say', 'It', 'back', '않다', 'right', 'down', 'but', '들', '널', 'at', '사랑', 'make', 'feel', '우리', 'We', 'never', 'way', 'Cause', '게', 'need', '수', 'night', 'So', '되다', 'if', 'are', 'see', '것', '한국어', '번역', '같다', '만', 've', 'come', '넌', 'how', 'ah', '해', 'ain', '밤', 'girl', 'life', 'If', '못', '속', '안', '걸', 'take', 'they', 'been', 'Don', 'good', 'too', 'hey', 'Baby', '돼다', '!', '눈', 'could', 'When', '나르다', '맘', 'away', '싶다', 'off', 'think', '한', '처럼', '또', 'u

In [61]:
import gensim

In [24]:
ko_model = gensim.models.Word2Vec.load('./cc.ko.300.bin')

UnpicklingError: invalid load key, '\xba'.

In [116]:
sum = 0
tokenized_query = Okt().morphs(query, stem=True)
div = len(query)
for word in tokenized_query:
    print(word)
    try: sum += model.wv[word]
    except: div = div - 1
sum /= div

여름
이라
너무
덥다


In [117]:
from sklearn.metrics.pairwise import cosine_similarity
cos_scores = []
for s in tqdm(w2v_nopre_vec):
    cos_scores.append(cosine_similarity(s.reshape(1, -1), sum.reshape(1, -1)))
index = np.argmax(cos_scores)
print(df.iloc[index])

100%|██████████| 3348/3348 [00:00<00:00, 6814.53it/s]

artist_names                                             YOUNGTAK
track_name                                                Jjiniya
lyrics          찐찐찐찐 찐이야 (찐) 찐찐찐찐 찐이야 완전 찐이야 진짜가 나타났다 지금 찐찐찐찐 ...
r                                                            70.0
g                                                            30.0
b                                                            97.0
Name: 1444, dtype: object


In [39]:
print(np.shape(w2v_nopre_vec))

(1674, 100)


## Tf-Idf를 이용

In [118]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
corpus.append(query)
tfidf_matrix = tfidf.fit_transform(corpus)

In [119]:
cos_scores = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [120]:
print(np.shape(cos_scores))

(1676, 1676)


In [121]:
print(cos_scores[-10:len(corpus)-1][:10])

[[0.02433803 0.00560394 0.05035088 ... 0.00767954 0.00150002 0.        ]
 [0.06450444 0.00910324 0.06912155 ... 0.01522773 0.00230519 0.        ]
 [0.01627667 0.00273938 0.00290207 ... 0.00494286 0.         0.        ]
 ...
 [0.07817843 0.01673785 0.15534557 ... 0.01779789 0.         0.        ]
 [0.01996154 0.00777241 0.04027673 ... 1.         0.00155191 0.        ]
 [0.         0.         0.00816413 ... 0.00155191 1.         0.        ]]


In [123]:
sim_scores = cos_scores[1675]
print(sim_scores)
sim_scores[1675] = -float('inf')
max_idx = sim_scores.argmax()
print(df.iloc[max_idx])

[0. 0. 0. ... 0. 0. 1.]
artist_names                                                  BTS
track_name                                                 Danger
lyrics          You in danger You in danger You in danger You ...
r                                                            25.0
g                                                            25.0
b                                                            25.0
Name: 1198, dtype: object


In [91]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Example corpus and query
corpus = ["The cat sat on the mat", "Dogs are great pets", "The sun is bright"]
query = "Cats and dogs"

# Ensure preprocessing consistency
tfidf = TfidfVectorizer(lowercase=True, stop_words='english')

# Add query to corpus
corpus.append(query)

# Transform into TF-IDF matrix
tfidf_matrix = tfidf.fit_transform(corpus)

# Calculate cosine similarity
cos_scores = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Check similarity of query (last document) with the rest
print(cos_scores[len(corpus) - 1])


[0.         0.30147576 0.         1.        ]
